<a href="https://colab.research.google.com/github/kahnertk/retinal-vessel-segmentation/blob/main/model_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#|default_export app

# Model deployment to Hugging Face Spaces using Gradio

In [88]:
## Install required packages

In [4]:
# NB: You can safely ignore any warnings or errors pip spits out about running as root or incompatibilities
!pip install -Uqq fastai
!pip install nbdev gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
  

## Load required packages

In [15]:
#|export
from fastai.vision.all import *
import matplotlib.pyplot as plt

## Define required functions

In [6]:
#|export
def get_y_fn(img_path):
    '''
    Function for the DataLoaders to grab the corresponding mask to each image.
    Takes the path of an image as input and returns the path of the corresponding mask.
    '''
    if "DRIVE" in img_path.stem:
        mask = data/'Training'/'Masks'/f'{img_path.stem}.gif'
    elif "CHASE" in img_path.stem:
        mask =  data/'Training'/'Masks'/f'{img_path.stem}.png'
    elif "HRF" in img_path.stem:
        mask =  data/'Training'/'Masks'/f'{img_path.stem}.tif'
    else:
        pass

    return mask


def segment(img):
    mask, *_ = learn.predict(img)
    #mask = plt.imshow(mask, cmap="gray")
    mask = np.array(mask) * 255
    return mask

## Load the model

In [7]:
#|export
learn = load_learner('vessel_seg.pkl')

## Build Gradio app

In [86]:
#|export
import gradio as gr
image = gr.Image()
mask = gr.Image(label="Predicted segmentation mask",
                         image_mode="L"
                         )
examples = ['04_test.png', '10_test.png', '19_test.png']
title = "Retinal vessel segmentation model v1"
desc = """Deep learning retinal vessel segmentation model using fastai's unet_learner model with a pretrained resnet34 architecture.
          The notebook for model creation and training can be found on kaggle: https://www.kaggle.com/kahnertk/retina-vessel-segmentation/"""
long_desc = """To train the model I used data from three published retina image data sets which have been combined and uploaded to Kaggle by Noob Data Scientist.
               Link to the combined data set:
               https://www.kaggle.com/datasets/pradosh123/retinal-vessel-segmentation-combined
               Links to the original data sets:
               DRIVE: https://paperswithcode.com/dataset/drive
               STARE: https://paperswithcode.com/dataset/stare
               CHASE_DB1: https://paperswithcode.com/dataset/chase-db1
               Example images are taken from the DRIVE data set"""

intf = gr.Interface(fn=segment,
                    inputs=image,
                    outputs=mask,
                    examples=examples,
                    title=title,
                    description=desc,
                    article=long_desc)
intf.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://411bdc8cee16df675a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
